In [1]:
from gru_lib import GRUModel, Pipeline
import tech_lib as tech
import binance_api as binance
%load_ext autoreload
%autoreload 2
import itertools
import pandas as pd
import numpy as np
import os
from datetime import datetime
import torch
print(f'Cuda available: {torch.cuda.is_available()}')
print(f'Cuda version: {torch.version.cuda}')
print(f'Cuda device name: {torch.cuda.get_device_name(0)}')

Cuda available: True
Cuda version: 12.6
Cuda device name: NVIDIA GeForce RTX 4060


In [2]:
api_key = 'lmxX1ymc6vGTQYkgUR194NeUxrhZQXM2xnf0rvjtQV4XxgP9xS9rawF6CvQRO2pw'
api_secret = 'rNFCb6OYjktZ1O6yfpNy1EItYJGwOMe8rtW8salpdXDHaNQGdxJDwpqgXrysxjN7'

ticker = 'BTCUSDT'
interval = '1m'
start_date = (2024, 1, 15, 0, 0, 0)
end_date = (2025, 1, 15, 0, 0, 0)

start_date_str = datetime(*start_date).strftime('%Y%m%d')
end_date_str = datetime(*end_date).strftime('%Y%m%d')
dataset_path = os.getcwd()+f'\\dataset\\{ticker}-{interval}-{start_date_str}-{end_date_str}.csv'

if os.path.exists(dataset_path):
    dataset = pd.read_csv(dataset_path, index_col='Open Time')
else:
    dataset = binance.historical_data(
        api_key,
        api_secret,
        ticker,
        interval,
        start_date,
        end_date
    )
    dataset = dataset.drop(['Ignore', 'Close Time'], axis=1)

dataset.to_csv(dataset_path)

In [3]:
#Calcul des outputs
dataset['Close_target'] = dataset.Close - dataset.Close.diff(-1)
#dataset['Returns'] = np.log(dataset['Close'] / dataset['Close'].shift(1))
#dataset['Returns_target'] = dataset.Returns - dataset.Returns.diff(-1)

# Boucle pour calculer les indicateurs avec différentes fenêtres
for window in range(30, 121, 30):
    dataset[f'cmf_{window}'] = tech.cmf(high=dataset.High, low=dataset.Low, close=dataset.Close, volume=dataset.Volume, window=window)
    dataset[f'rsi_{window}'] = tech.rsi(close=dataset.Close, window=window)
    dataset[f'ma_{window}'] = tech.ma(close=dataset.Close, window=window)
    dataset[f'ema_{window}'] = tech.ema(close=dataset.Close, window=window)
    dataset[f'atr_{window}'] = tech.atr(high=dataset.High, low=dataset.Low, close=dataset.Close, window=window)
    dataset[f'bollinger_{window}'] = tech.bollinger_band_width(close=dataset.Close, window=window, num_std=2)
dataset['obv'] = tech.obv(dataset.Close, dataset.Volume)

dataset = dataset.dropna()

In [4]:
dataset.tail()

,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Close_h1,...,obv,Close_target,atr_30,bollinger_30,atr_60,bollinger_60,atr_90,bollinger_90,atr_120,bollinger_120
Open Time,,,,,,,,,,,,,,,,,,,,,
14/01/2025 23:54,96607.35,96655.68,96607.35,96624.09,4.74550,458546.5070,1281,2.96589,286564.83460,96607.65,...,-25819.339307,96607.65,35.258505,245.327426,35.980034,277.527579,38.121498,242.577928,40.956192,292.983399
14/01/2025 23:55,96624.09,96624.10,96607.65,96607.65,2.52772,244234.2473,431,1.04096,100581.80410,96606.51,...,-25821.867027,96606.51,34.045053,245.523318,35.339705,272.690709,37.645202,242.444412,40.551131,282.958478
14/01/2025 23:56,96607.66,96607.66,96606.51,96606.51,1.37214,132557.9608,247,0.11912,11507.77491,96541.30,...,-25823.239167,96541.30,31.922791,245.412246,34.218731,267.536979,36.843109,242.420587,39.899872,279.332287
14/01/2025 23:57,96606.51,96606.51,96541.29,96541.30,4.67059,451071.8629,1010,0.75549,72965.35194,96536.96,...,-25827.909757,96536.96,34.070998,246.331521,35.235166,269.154760,37.466777,244.968990,40.318387,272.918849
14/01/2025 23:58,96541.29,96549.71,96536.95,96536.96,3.48486,336434.5289,527,2.03919,196868.07520,96560.86,...,-25831.394617,96560.86,32.696095,245.893862,34.498275,264.371065,36.923771,247.655970,39.862876,268.497770


In [7]:
from gru_lib import GRUModel, Pipeline
%load_ext autoreload
%autoreload 2

# Définir les plages de valeurs pour les hyperparamètres
param_grid = {
    "period_size"   : [50000],
    "hidden_size"   : [8],
    "num_layers"    : [1],
    "dropout"       : [0.0, 0.1],
    "lr"            : [1],
    "batch_size"    : [100, 200],
    "epochs"        : [5],
    "train_size"    : [0.7],
    "val_size"      : [0.15],
    "test_size"     : [0.15],
    "seq_size"      : [60, 120]
}

inputs = [
    'Open', 'Close', 'High', 'Low', 
    'Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
    'obv', 'cmf_30', 'ema_30', 'rsi_30', 'ma_30', 'bollinger_30', 'atr_30'
]
min_imputs = 6
outputs = 'Close_target'

result_path = os.getcwd()+f'\\train_result\\GRU-{ticker}-{interval}.csv'

# Générer toutes les combinaisons possibles d'hyperparamètres
param_combinations = list(itertools.product(*param_grid.values()))

# Générer toutes les combinaisons additives possibles des inputs
input_combinations = [list(itertools.combinations(inputs, r)) for r in range(min_imputs, len(inputs) + 1)]
input_combinations = [item for sublist in input_combinations for item in sublist]

counter = 0
for params, input_subset in itertools.product(param_combinations, input_combinations):
    time_lauch = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'---------- Launch Time: {time_lauch:<10} ---------- Progress: {counter}/{len(param_combinations) * len(input_combinations):<3} ----------')

    # Extraire les paramètres pour cette combinaison
    hyperparams = dict(zip(param_grid.keys(), params))

    # Garder la période étudiée
    df = dataset.iloc[-hyperparams["period_size"]:]
    df = df.dropna()

    # Initialiser le modèle
    model = GRUModel(
        input_size=len(input_subset),
        output_size=1,
        hidden_size=hyperparams["hidden_size"], 
        num_layers=hyperparams["num_layers"],
        dropout=hyperparams["dropout"]
    )

    # Initialiser le pipeline
    pipeline = Pipeline(
        model=model, 
        dataset=df,
        inputs=list(input_subset),
        outputs=outputs
    )

    # Définir les hyperparamètres
    pipeline.hyper_param(
        lr=hyperparams["lr"], 
        batch_size=hyperparams["batch_size"], 
        epochs=hyperparams["epochs"]
    )

    # Prétraitement des données
    pipeline.preprocess(
        train_size=hyperparams["train_size"],
        val_size=hyperparams["val_size"], 
        test_size=hyperparams["test_size"], 
        seq_size=hyperparams["seq_size"]
    )

    pipeline.train()
    pipeline.eval()
    pipeline.save(result_path)

    print("\n")
    counter += 1


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
---------- Launch Time: 2025-01-19 21:38:31 ---------- Progress: 0/103288 ----------
Epoch 1/5, Train Loss: 8,999,100,357.445562, Validation Loss: 41,855,390,173.140427, Time: 0:00:02.538162
Epoch 2/5, Train Loss: 8,010,219,248.775846, Validation Loss: 37,256,041,016.646805, Time: 0:00:02.345878


KeyboardInterrupt: 